In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TestApp").getOrCreate()

In [0]:
df = spark.read.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/LLCP2024.parquet', header=True, inferSchema=True)

df.printSchema()

In [0]:
df.show()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

# subset data to plot
df_subset = df.sample(True, 0.01, seed=123)
df_subset = df_subset.toPandas()

sns.countplot(df_subset, x="GENHLTH")
# 7 and 9 are rare and are basically unknown

In [0]:
# note that high values of POORHLTH
df_subset['POORHLTH_1'] = df_subset['POORHLTH'].apply(lambda x: 0 if x == 88 else x) # 88 means NONE
sns.histplot(df_subset[df_subset['POORHLTH_1']<=31], x="POORHLTH_1")

In [0]:
sns.countplot(df_subset, x="_AGEG5YR")

In [0]:
sns.countplot(df_subset, x="MARITAL")

In [0]:
sns.countplot(df_subset, x="EDUCA")

In [0]:
sns.countplot(df_subset, x="RENTHOM1")

In [0]:
fig, ax =plt.subplots(1,2)
sns.histplot(df_subset, x="_BMI5", ax=ax[0])
sns.countplot(df_subset, x="_BMI5CAT", ax=ax[1])
fig.show()

In [0]:
fig, ax =plt.subplots(1,2)
sns.countplot(df_subset[~(df_subset['INCOME3'].isin(list([77, 99])))], x="INCOME3", ax=ax[0]) # 77, 99 unknown
sns.countplot(df_subset[df_subset['_INCOMG1']!=9], x="_INCOMG1", ax=ax[1])
plt.subplots_adjust(wspace=0.3)
fig.show()

In [0]:
df_subset['num_conditions']=(df_subset[['CVDINFR4','CVDCRHD4','CVDSTRK3','ASTHMA3','CHCSCNC1','CHCOCNC1','CHCCOPD3','ADDEPEV3','CHCKDNY2','HAVARTH4','DIABETE4']]==1).sum(axis=1)

In [0]:
sns.countplot(df_subset, x="num_conditions")

In [0]:
df_subset.columns

In [0]:
sns.countplot(df_subset, x="_STATE")
plt.xticks(rotation=45, size=6)
plt.show()

In [0]:
fig, ax =plt.subplots(1,2)
sns.countplot(df_subset[~(df_subset['CHILDREN'].isin(list([77, 99])))], x="CHILDREN", ax=ax[0]) # 77, 99 unknown
plt.xticks(rotation=45, size=6)
sns.countplot(df_subset[~(df_subset['HHADULT'].isin(list([77, 88,99])))], x="HHADULT", ax=ax[1]) # 77, 99 unknown
plt.subplots_adjust(wspace=0.3)
plt.xticks(rotation=45, size=6)
fig.show()

In [0]:
import numpy as np
def modify_children(x):
    if x==88:
        return 0
    elif x ==99:
        return np.nan
    else:
        return min(8,x)
df_subset['CHILDREN_mod']=[modify_children(x) for x in df_subset['CHILDREN']]
sns.countplot(df_subset, x="CHILDREN_mod")

In [0]:
def modify_adults(x):
    if x in [0,77,88,99]:
        return np.nan
    else:
        return min(9,x)
df_subset['ADULT_mod']=[modify_adults(x) for x in df_subset['HHADULT']]
sns.countplot(df_subset, x="ADULT_mod")

In [0]:
import pandas as pd
poverty_df = pd.read_csv('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/thresh24.csv')
poverty_df=poverty_df.rename(columns={'household':'ADULT_mod', 'children':'CHILDREN_mod','threshold':'poverty_threshold'})


In [0]:
bins=[0,10000,15000,20000,25000,35000,50000,75000,100000,150000,200000,np.inf]
poverty_df['poverty_threshold_conv']=pd.cut(poverty_df['poverty_threshold'],bins, labels=[x for x in range(1,12)])
poverty_df['poverty_threshold_conv']=pd.factorize(poverty_df['poverty_threshold_conv'])[0] + 2

In [0]:
poverty_df

In [0]:
df_subset[['ADULT_mod','CHILDREN_mod']]

In [0]:
df_subset=df_subset.merge(poverty_df, on=['ADULT_mod','CHILDREN_mod'])

In [0]:
def remove_7799(x):
    if x in[77,99]:
        return np.nan
    else:
        return x
df_subset['INCOME3_adj']=[remove_7799(x) for x in df_subset['INCOME3']]
df_subset['income_adj_pov'] = df_subset['INCOME3_adj']-df_subset['poverty_threshold_conv']
sns.countplot(df_subset, x="income_adj_pov")
plt.xticks(rotation=45, size=6)

In [0]:
df_subset[['INCOME3','poverty_threshold_conv']].dtypes